In [1]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as pyplot
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
# from tensorflow.keras

In [ ]:
import os
import pandas as pd

# Define the path to the training folder
train_dir = "train"

data = []

# Traverse the directory
for label in os.listdir(train_dir):
    label_folder = os.path.join(train_dir, label)
    if os.path.isdir(label_folder):
        for img_file in os.listdir(label_folder):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(label_folder, img_file)
                data.append({'image_path': img_path, 'label': label})
                            
df = pd.DataFrame(data)

print(df.head())

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\train'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define Directory Paths
train_dir = r"C:\Users\Velpr\Documents\Fish\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\train"
test_dir = r"C:\Users\Velpr\Documents\Fish\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\test"
val_dir = r"C:\Users\Velpr\Documents\Fish\Dataset\images.cv_jzk6llhf18tm3k0kyttxz\data\val"

# Set Image Parameters
IMG_SIZE = (292, 292)
BATCH_SIZE = 32

# Data Augmentation for Training Set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,           # rotate image up to 20 degrees
    zoom_range=0.2,              # zoom in/out by 20%
    width_shift_range=0.2,       # shift image left/right
    height_shift_range=0.2,      # shift image up/down
    horizontal_flip=True,        # flip image horizontally
    fill_mode='nearest'          # fill empty pixels after rotation/shift
)

In [ ]:
# Only rescaling for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load validation data
val_generator = val_test_datagen.flow_from_directory(
    directory=val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load test data
test_generator = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Setup
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory=test_dir, 
    target_size=(128, 128),
    batch_size=1,
    class_mode='categorical'  
)

# Plot 6 images from the generator
fig, ax = plt.subplots(2, 3, figsize=(12, 8))
ax = ax.flatten()

for j in range(6):
    img, label = next(test_generator)
    ax[j].imshow(img[0])
    label_idx = label[0].argmax()  
    class_name = list(test_generator.class_indices.keys())[label_idx]
    ax[j].set_title(class_name)
    ax[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(292, 292, 3)))  
model.add(MaxPooling2D(pool_size=(2, 2)))                                     

# Second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to feed into Dense layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) 

# Output layer
model.add(Dense(train_generator.num_classes, activation='softmax'))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Match model output to number of classes
num_classes = train_generator.num_classes

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(292, 292, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(num_classes, activation='softmax') 
])
    

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, validation_data=val_generator, epochs=10)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy*100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
model = Sequential([
    Flatten(input_shape=(128, 128, 3)),  # This flattens the images from the generator
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])


# Predict on test data
y_probs = model.predict(test_generator)
y_pred = np.argmax(y_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print detailed report
report = classification_report(y_true, y_pred, target_names=class_labels)
print("Classification Report of CNN:\n", report)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix of CNN:\n", cm)

In [ ]:
# 5 Pre-Model Training

from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
Num_classes = 11
Epochs = 5
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Load base model (exclude top)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False  # Freeze base layers

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(Num_classes, activation='softmax')(x)

model_vgg16 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_vgg16.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Save only best model
checkpoint = ModelCheckpoint('vgg16_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Train
history_vgg16 = model_vgg16.fit(
    train_generator,
    validation_data=val_generator,
    epochs=Epochs,
    callbacks=[checkpoint, early_stop]
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np

# Set the correct target size
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    r"C:\Users\pavit\OneDrive\Documents\MULTICLASS_IMAGE\test",                   # <-- your actual path
    target_size=(292, 292),       # <-- must match training size!
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
# Load best model
model_vgg16 = tf.keras.models.load_model('vgg16_model.h5')

# Predict on test set
Y_pred = model_vgg16.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification report
report_vgg = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
print("Classification Report for VGG16:\n", classification_report(y_true, y_pred, target_names=class_labels))

# Confusion matrix
conf_mat = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_mat)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(), cmap='Blues')
plt.title('VGG16 Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.plot(history_vgg16.history['accuracy'], label='Train Acc')
plt.plot(history_vgg16.history['val_accuracy'], label='Val Acc')
plt.title('VGG16 Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history_vgg16.history['loss'], label='Train Loss')
plt.plot(history_vgg16.history['val_loss'], label='Val Loss')
plt.title('VGG16 Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout
plt.show

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Load base model
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model_resnet.trainable = False  # Freeze pre-trained layers

# Add custom top layers
x = base_model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(Num_classes, activation='softmax')(x)

model_resnet = Model(inputs=base_model_resnet.input, outputs=x)

model_resnet.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

checkpoint_resnet = ModelCheckpoint("resnet_model.h5", save_best_only=True, monitor='val_accuracy', mode='max')
early_stop_resnet = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history_resnet = model_resnet.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint_resnet, early_stop_resnet]
    )

In [ ]:
# Predict on test set
Y_pred = model_resnet.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Classification Report
report_resnet = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
print("Classification Report for ResNet50:")
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion Matrix
cm = confusion_matrix(test_generator.classes, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(), cmap='Reds')
plt.title('ResNet50 Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_resnet.history['accuracy'], label='Train Accuracy')
plt.plot(history_resnet.history['val_accuracy'], label='Val Accuracy')
plt.title('ResNet50 Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_resnet.history['loss'], label='Train Loss')
plt.plot(history_resnet.history['val_loss'], label='Val Loss')
plt.title('ResNet50 Loss')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

base_model_mobile = MobileNet(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model_mobile.trainable = False  # Freeze base layers

# Add custom top layers
x = base_model_mobile.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(Num_classes, activation='softmax')(x)

model_mobile = Model(inputs=base_model_mobile.input, outputs=output)

# Compile the model
model_mobile.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

checkpoint_mobile = ModelCheckpoint("mobilenet_model.h5", save_best_only=True, monitor='val_accuracy', mode='max')
early_stop_mobile = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history_mobile = model_mobile.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint_mobile, early_stop_mobile]
)

In [ ]:
Y_pred_mobile = model_mobile.predict(test_generator)
y_pred_mobile = np.argmax(Y_pred_mobile, axis=1)

# Classification report
report_mobilenet = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
print("Classification Report for MobileNet:")
print(classification_report(test_generator.classes, y_pred_mobile, target_names=test_generator.class_indices.keys()))

# Confusion matrix
cm_mobile = confusion_matrix(test_generator.classes, y_pred_mobile)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_mobile, annot=True, fmt='d',
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(),
            cmap='Greens')
plt.title('MobileNet Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_mobile.history['accuracy'], label='Train Accuracy')
plt.plot(history_mobile.history['val_accuracy'], label='Val Accuracy')
plt.title('MobileNet Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_mobile.history['loss'], label='Train Loss')
plt.plot(history_mobile.history['val_loss'], label='Val Loss')
plt.title('MobileNet Loss')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Load pre-trained InceptionV3 without top
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model_inception.trainable = False  # Freeze base layers

# Add custom top layers
x = base_model_inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(Num_classes, activation='softmax')(x)

model_inception = Model(inputs=base_model_inception.input, outputs=output)

# Compile the model
model_inception.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

checkpoint_inception = ModelCheckpoint("inception_model.h5", save_best_only=True, monitor='val_accuracy', mode='max')
early_stop_inception = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history_inception = model_inception.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[checkpoint_inception, early_stop_inception]
)

In [ ]:
# Predict test set
Y_pred_inception = model_inception.predict(test_generator)
y_pred_inception = np.argmax(Y_pred_inception, axis=1)

# Classification report
report_inception = classification_report(y_true, y_pred_inception, target_names=class_labels, output_dict=True)
print("Classification Report for InceptionV3:")
print(classification_report(test_generator.classes, y_pred_inception, target_names=test_generator.class_indices.keys()))

# Confusion matrix
cm_inception = confusion_matrix(test_generator.classes, y_pred_inception)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_inception, annot=True, fmt='d',
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(),
            cmap='Oranges')
plt.title('InceptionV3 Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_inception.history['accuracy'], label='Train Accuracy')
plt.plot(history_inception.history['val_accuracy'], label='Val Accuracy')
plt.title('InceptionV3 Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_inception.history['loss'], label='Train Loss')
plt.plot(history_inception.history['val_loss'], label='Val Loss')
plt.title('InceptionV3 Loss')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Load pre-trained EfficientNetB0 without top
base_model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model_efficientnet.trainable = False  # Freeze base layers

# Add custom top layers
x = base_model_efficientnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(Num_classes, activation='softmax')(x)

model_efficientnet = Model(inputs=base_model_efficientnet.input, outputs=output)

# Compile the model
model_efficientnet.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
checkpoint_efficientnet = ModelCheckpoint("efficientnet_model.h5", save_best_only=True, monitor='val_accuracy', mode='max')
early_stop_efficientnet = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history_efficientnet = model_efficientnet.fit(
    train_generator,
    validation_data=val_generator,
    epochs=Epochs,
    callbacks=[checkpoint_efficientnet, early_stop_efficientnet]
)

In [ ]:
# Predict test set
Y_pred_efficientnet = model_efficientnet.predict(test_generator)
y_pred_efficientnet = np.argmax(Y_pred_efficientnet, axis=1)

# Classification report
report_effnet = classification_report(y_true, y_pred_efficientnet, target_names=class_labels, output_dict=True)
print("Classification Report for EfficientNetB0:")
print(classification_report(test_generator.classes, y_pred_efficientnet, target_names=test_generator.class_indices.keys()))

# Confusion matrix
cm_efficientnet = confusion_matrix(test_generator.classes, y_pred_efficientnet)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_efficientnet, annot=True, fmt='d',
            xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(),
            cmap='Purples')
plt.title('EfficientNetB0 Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_efficientnet.history['accuracy'], label='Train Accuracy')
plt.plot(history_efficientnet.history['val_accuracy'], label='Val Accuracy')
plt.title('EfficientNetB0 Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_efficientnet.history['loss'], label='Train Loss')
plt.plot(history_efficientnet.history['val_loss'], label='Val Loss')
plt.title('EfficientNetB0 Loss')
plt.legend()
plt.show()

In [ ]:
import pandas as pd

# Combine key metrics (weighted avg) from each report
summary = {
    'Model': ['VGG16', 'ResNet50', 'MobileNet', 'InceptionV3', 'EfficientNetB0'],
    'Accuracy': [
        report_vgg['accuracy'],
        report_resnet['accuracy'],
        report_mobilenet['accuracy'],
        report_inception['accuracy'],
        report_effnet['accuracy']
    ],
    'Precision': [
        report_vgg['weighted avg']['precision'],
        report_resnet['weighted avg']['precision'],
        report_mobilenet['weighted avg']['precision'],
        report_inception['weighted avg']['precision'],
        report_effnet['weighted avg']['precision']
    ],
    'Recall': [
        report_vgg['weighted avg']['recall'],
        report_resnet['weighted avg']['recall'],
        report_mobilenet['weighted avg']['recall'],
        report_inception['weighted avg']['recall'],
        report_effnet['weighted avg']['recall']
    ],
    'F1-Score': [
        report_vgg['weighted avg']['f1-score'],
        report_resnet['weighted avg']['f1-score'],
        report_mobilenet['weighted avg']['f1-score'],
        report_inception['weighted avg']['f1-score'],
        report_effnet['weighted avg']['f1-score']
    ]
}

df_summary = pd.DataFrame(summary)
print(df_summary)

In [ ]:
import json

with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)